In [1]:
# Modules to support development
import os
import re
import collections
import itertools
import functools
import logging
import pprint
import numpy as np
import heapq
import copy
from tqdm import tqdm
import math

In [2]:
def read_input(puzzle_input):
    with open(puzzle_input) as ff:
        dd = [ ll.replace("\n", "") for ll in ff.readlines() ]

    width = max([ len(ll) for ll in dd])
    height = len(dd) - 2
    print(height, width)

    starting_pos = (height, width)
    map = np.zeros((height, width))
    for yy, ll in enumerate(dd[0:-2]):
        for xx, cc in enumerate(ll):
            if cc == '#':
                map[yy,xx] = -1
            elif cc == ".":
                map[yy,xx] = 1
                if (yy, xx) < starting_pos:
                    starting_pos = (yy, xx)

    
    notes = []
    ii = 0
    accum = ''
    while ii < len(dd[-1].strip()):
        if dd[-1][ii].isdecimal():
            accum += dd[-1][ii]
        elif dd[-1][ii] in ('L', 'R'):
            notes.append(int(accum))
            notes.append(dd[-1][ii])
            accum = ''
        ii += 1
    if accum:
        notes.append(int(accum))

    return map, notes, starting_pos

def test_read_input():
    map, notes, starting_pos = read_input(os.path.join(os.path.join("..", "dat", "day22_test.txt")))
    print(map)
    print(notes)
    print(starting_pos)

test_read_input()

12 16
[[ 0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1. -1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1. -1.  1.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. -1.  1.  1.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  0.  0.  0.  0.]
 [ 1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1. -1.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  0.  0.  0.  0.]
 [ 1.  1. -1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1. -1.  1.  1.  1.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1. -1.  1.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1. -1.  1.  1.  1.  1.  1.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1. -1.  1.]]
[10, 'R', 5, 'L', 5, 'R', 10, 'L', 4, 'R', 5, 'L', 5]
(0, 8)


In [3]:
rotate = {
    'L': { '>': '^', '^': '<', '<': 'v', 'v': '>'},
    'R': { '>': 'v', 'v': '<', '<': '^', '^': '>'},
}
points = {'>': 0, 'v': 1, '<': 2, '^': 3}

def part1(puzzle_input):
    map, notes, starting_pos = read_input(puzzle_input)
    print(notes[-3:])
    current_pos = (starting_pos[0], starting_pos[1], '>')
    print("Starting at", current_pos)
    for action in notes:
        if action in ('R', 'L'):
            current_pos = (
                current_pos[0],
                current_pos[1],
                rotate.get(action).get(current_pos[2])
            )
        else:
            if current_pos[2] == '>': # move right
                delta = (0, 1)
            elif current_pos[2] == '<': # move left
                delta = (0, -1)
            elif current_pos[2] == '^': # move up
                delta = (-1, 0)
            elif current_pos[2] == 'v': # move down
                delta = (1, 0)

            next_pos = current_pos
            while action > 0:
                next_pos = [
                    next_pos[0] + delta[0],
                    next_pos[1] + delta[1],
                    next_pos[2]
                ]
                
                if next_pos[0] < 0:
                    # wrap around from top to bottom
                    next_pos[0] = map.shape[0] - 1
                elif next_pos[0] >= map.shape[0]:
                    # wrap around from bottom to top
                    next_pos[0] = 0

                if next_pos[1] < 0:
                    # wrap around from left to right
                    next_pos[1] = map.shape[1] - 1
                elif next_pos[1] >= map.shape[1]:
                    # wrap around from right to left
                    next_pos[1] = 0

                if map[next_pos[0], next_pos[1]] == -1:
                    # we hit a wall
                    break
                elif map[next_pos[0], next_pos[1]] == 1:
                    # we hit an open space
                    action -= 1
                    current_pos = next_pos

            print("now at", current_pos)

    print("Final Position", current_pos)
    return (current_pos[0] + 1) * 1000 + ((current_pos[1] + 1) * 4) + points.get(current_pos[2])

        

def test_part1():
    ans = part1(os.path.join(os.path.join("..", "dat", "day22_test.txt")))
    print(ans)
    assert ans == 6032

test_part1()

ans = part1(os.path.join(os.path.join("..", "dat", "day22.txt")))
print(ans) # 162186


12 16
[5, 'L', 5]
Starting at (0, 8, '>')
now at [0, 10, '>']
now at [5, 10, 'v']
now at [5, 3, '>']
now at [7, 3, 'v']
now at [7, 7, '>']
now at [5, 7, 'v']
now at (5, 7, '>')
Final Position (5, 7, '>')
6032
200 5631
[26, 'L', 50]
Starting at (0, 50, '>')
now at [0, 67, '>']
now at (0, 67, 'v')
now at (0, 67, '>')
now at (0, 67, 'v')
now at [0, 50, '<']
now at (0, 50, 'v')
now at [0, 51, '>']
now at [4, 51, 'v']
now at [4, 137, '<']
now at [19, 137, 'v']
now at [19, 149, '>']
now at [29, 149, 'v']
now at [29, 131, '<']
now at [16, 131, '^']
now at [16, 139, '>']
now at [24, 139, 'v']
now at [24, 52, '>']
now at [39, 52, 'v']
now at [39, 134, '<']
now at [40, 134, 'v']
now at [40, 117, '<']
now at [5, 117, 'v']
now at [5, 125, '>']
now at [44, 125, '^']
now at [44, 132, '>']
now at [28, 132, '^']
now at [28, 126, '<']
now at [32, 126, 'v']
now at [32, 116, '<']
now at (32, 116, 'v')
now at [32, 132, '>']
now at [28, 132, '^']
now at [28, 131, '<']
now at [16, 131, '^']
now at [16, 139,

In [83]:
def what_face_test(pos, face_size):
    if   (0 <= pos[0] < face_size) and (2*face_size <= pos[1] < 3*face_size):
        face = 1
    elif (face_size <= pos[0] < 2*face_size) and (0 <= pos[1] < face_size):
        face = 2
    elif (face_size <= pos[0] < 2*face_size) and (face_size <= pos[1] < 2*face_size):
        face = 3
    elif (face_size <= pos[0] < 2*face_size) and (2*face_size <= pos[1] < 3*face_size):
        face = 4
    elif (2*face_size <= pos[0] < 3*face_size) and (2*face_size <= pos[1] < 3*face_size):
        face = 5
    elif (2*face_size <= pos[0] < 3*face_size) and (3*face_size <= pos[1] < 4*face_size):
        face = 6
    else:
        face = None
        
    return face

def calc_next_pos_test_input(current_pos, face_size):
    if current_pos[2] == '>': # move right
        delta = (0, 1)
    elif current_pos[2] == '<': # move left
        delta = (0, -1)
    elif current_pos[2] == '^': # move up
        delta = (-1, 0)
    elif current_pos[2] == 'v': # move down
        delta = (1, 0)

    next_pos = [
        current_pos[0] + delta[0],
        current_pos[1] + delta[1],
        current_pos[2]
    ]

    if next_pos[0] < 0:
        face = (2,3,1,6)[next_pos[1] // face_size]
        facecol = next_pos[1] % face_size

        if face == 2:
            next_pos[0] = 0
            next_pos[1] = (face_size * 3) - facecol - 1 
            next_pos[2] = 'v'
            assert what_face_test(next_pos, face_size) == 1
        elif face == 3:
            next_pos[0] = facecol
            next_pos[1] = face_size * 2
            next_pos[2] = '>'
            assert what_face_test(next_pos, face_size) == 1
        elif face == 1:
            next_pos[0] = face_size
            next_pos[1] = face_size - facecol - 1
            next_pos[2] = 'v'
            assert what_face_test(next_pos, face_size) == 2
        elif face == 6:
            next_pos[0] = (face_size * 2) - facecol - 1
            next_pos[1] = (face_size * 3) - 1
            next_pos[2] = '<'
            assert what_face_test(next_pos, face_size) == 4
    elif next_pos[0] >= (face_size*3):
        face = (2,3,5,6)[next_pos[1] // face_size]
        facecol = next_pos[1] % face_size
        if face == 2:
            next_pos[0] = (face_size*4) - 1
            next_pos[1] = (face_size*4) - facecol - 1
            next_pos[2] = '^'
            assert what_face_test(next_pos, face_size) == 5, str(current_pos)
        elif face == 3:
            next_pos[0] = (face_size*3) - facecol - 1
            next_pos[1] = face_size * 2
            next_pos[2] = '>'
            assert what_face_test(next_pos, face_size) == 5
        elif face == 5:
            next_pos[0] = face_size*2 - 1
            next_pos[1] = face_size - facecol - 1
            next_pos[2] = '^'
            assert what_face_test(next_pos, face_size) == 2
        elif face == 6:
            next_pos[0] = (face_size * 2) - facecol - 1
            next_pos[1] = 0
            next_pos[2] = '>'
            assert what_face_test(next_pos, face_size) == 2

    if next_pos[1] < 0:
        face = (1,2,5)[next_pos[0] // face_size]
        facerow = next_pos[0] % face_size
        if face == 1:
            next_pos[0] = face_size
            next_pos[1] = face_size + facerow
            next_pos[2] = 'v'
            assert what_face_test(next_pos, face_size) == 3
        elif face == 2:
            next_pos[0] = face_size*3 - 1
            next_pos[1] = face_size*4 - facerow - 1
            next_pos[2] = '^'
            assert what_face_test(next_pos, face_size) == 6
        elif face == 5:
            next_pos[0] = face_size*2 - 1
            next_pos[1] = face_size*2 - facerow - 1
            next_pos[2] = '^'
            assert what_face_test(next_pos, face_size) == 3

    elif next_pos[1] >= (face_size*4):
        face = (1,4,6)[next_pos[0] // face_size]
        facerow = next_pos[0] % face_size
        if face == 1:
            next_pos[0] = (face_size * 3) - facerow - 1
            next_pos[1] = (face_size * 4) - 1
            next_pos[2] = '<'
            assert what_face_test(next_pos, face_size) == 6
        elif face == 4:
            next_pos[0] = face_size*2
            next_pos[1] = face_size*4 - facerow - 1
            next_pos[2] = 'v'
            assert what_face_test(next_pos, face_size) == 6
        elif face == 6:
            next_pos[0] = face_size - facerow - 1
            next_pos[1] = face_size*3 - 1
            next_pos[2] = '<'
            assert what_face_test(next_pos, face_size) == 1
    
    print(next_pos)
    assert 0 <= next_pos[0]
    assert next_pos[0] < face_size*3
    assert 0 <= next_pos[1]
    assert next_pos[1] < face_size*4
    
    return tuple(next_pos)

assert calc_next_pos_test_input((0,0,'^'), 4) == (0,11,'v')
assert calc_next_pos_test_input((0,3,'^'), 4) == (0,8,'v')
assert calc_next_pos_test_input((0,4,'^'), 4) == (0,8,'>')
assert calc_next_pos_test_input((0,7,'^'), 4) == (3,8,'>')
assert calc_next_pos_test_input((0,8,'^'), 4) == (4,3,'v')
assert calc_next_pos_test_input((0,11,'^'), 4) == (4,0,'v')
assert calc_next_pos_test_input((0,12,'^'), 4) == (7,11,'<')
assert calc_next_pos_test_input((0,15,'^'), 4) == (4,11,'<')
assert calc_next_pos_test_input((5,10,'>'), 4) == (5,11,'>')
assert calc_next_pos_test_input((5,11,'>'), 4) == (5,12,'>')
assert calc_next_pos_test_input((5,12,'>'), 4) == (5,13,'>')
assert calc_next_pos_test_input((11,10,'v'), 4) == (7,1,'^')
assert calc_next_pos_test_input((6,0,'<'), 4) == (11,13,'^')
assert calc_next_pos_test_input((10,15,'>'), 4) == (1,11,'<')
assert calc_next_pos_test_input((11,5,'v'), 4) == (10,8,'>')

[0, 11, 'v']
[0, 8, 'v']
[0, 8, '>']
[3, 8, '>']
[4, 3, 'v']
[4, 0, 'v']
[7, 11, '<']
[4, 11, '<']
[5, 11, '>']
[5, 12, '>']
[5, 13, '>']
[7, 1, '^']
[11, 13, '^']
[1, 11, '<']
[10, 8, '>']


In [111]:
def what_face(pos, face_size):
    row = pos[0] // face_size
    col = pos[1] // face_size

    faces = {
        (0, 1): 'A',
        (0, 2): 'B',
        (1, 1): 'C',
        (2, 0): 'E',
        (2, 1): 'D',
        (3, 0): 'F',
    }

    face = faces.get((row, col), None)
        
    return face

assert what_face((0,0), 50) == None
assert what_face((0,49), 50) == None
assert what_face((0,50), 50) == 'A'
assert what_face((0,99), 50) == 'A'
assert what_face((49,99), 50) == 'A'
assert what_face((0,100), 50) == 'B'
assert what_face((75,75), 50) == 'C'
assert what_face((149,75), 50) == 'D'
assert what_face((149, 25), 50) == 'E'
assert what_face((175, 25), 50) == 'F'
assert what_face((175, 75), 50) == None

def rel2abs(face, pos):
    face_ul_corners = {
        'A': (0, 50),
        'B': (0, 100),
        'C': (50, 50),
        'D': (100, 50),
        'E': (100, 0),
        'F': (150, 0),
    }

    ul = face_ul_corners[face]
    return (ul[0] + pos[0], ul[1] + pos[1])

def calc_next_pos(current_pos, face_size):
    if current_pos[2] == '>': # move right
        delta = (0, 1)
    elif current_pos[2] == '<': # move left
        delta = (0, -1)
    elif current_pos[2] == '^': # move up
        delta = (-1, 0)
    elif current_pos[2] == 'v': # move down
        delta = (1, 0)

    next_pos = [
        current_pos[0] + delta[0],
        current_pos[1] + delta[1],
        current_pos[2]
    ]

    if next_pos[0] < 0:
        # Gon
        face = ('E', 'A', 'B')[next_pos[1] // face_size]
        facecol = next_pos[1] % face_size

        if face == 'E':
            print("E->C")
            next_pos[0], next_pos[1] = rel2abs('C', (facecol, 0))
            next_pos[2] = '>'
            assert what_face(next_pos, face_size) == 'C'
        elif face == 'A':
            print("A->F")
            next_pos[0], next_pos[1] = rel2abs('F', (facecol, 0))
            next_pos[2] = '>'
            assert what_face(next_pos, face_size) == 'F'
        elif face == 'B':
            print("B->F")
            next_pos[0], next_pos[1] = rel2abs('F', (face_size-1, facecol))
            next_pos[2] = '^'
            assert what_face(next_pos, face_size) == 'F'
        
    elif next_pos[0] >= (face_size*4):
        face = ('F', 'D', 'B')[next_pos[1] // face_size]
        facecol = next_pos[1] % face_size
        if face == 'F':
            print("F->B")
            next_pos[0], next_pos[1] = rel2abs('B', (0, facecol))
            next_pos[2] = 'v'
            assert what_face(next_pos, face_size) == 'B'
        elif face == 'D':
            print("D->F")
            next_pos[0], next_pos[1] = rel2abs('F', (facecol, face_size-1))
            next_pos[2] = '<'
            assert what_face(next_pos, face_size) == 'F'
        elif face == 'B':
            print("B->C")
            next_pos[0], next_pos[1] = rel2abs('C', (facecol, face_size-1))
            next_pos[2] = '<'
            assert what_face(next_pos, face_size) == 'C'

    if next_pos[1] < 0:
        face = ('A', 'C', 'E', 'F')[next_pos[0] // face_size]
        facerow = next_pos[0] % face_size
        if face == 'A':
            print("A->E")
            next_pos[0], next_pos[1] = rel2abs('E', (face_size-facerow-1, 0))
            next_pos[2] = '>'
            assert what_face(next_pos, face_size) == 'E', f"{next_pos} {face_size} {facerow}"
        elif face == 'C':
            print("C->E")
            next_pos[0], next_pos[1] = rel2abs('E', (0, facerow))
            next_pos[2] = 'v'
            assert what_face(next_pos, face_size) == 'E'
        elif face == 'E':
            print("E->A")
            next_pos[0], next_pos[1] = rel2abs('A', (face_size-facerow-1, 0))
            next_pos[2] = '>'
            assert what_face(next_pos, face_size) == 'A', f"{next_pos} {face_size} {facerow}"
        elif face == 'F':
            print("F->A")
            next_pos[0], next_pos[1] = rel2abs('A', (0, facerow))
            next_pos[2] = 'v'
            assert what_face(next_pos, face_size) == 'A'

    elif next_pos[1] >= (face_size*3):
        face = ('B', 'C', 'D', 'F')[next_pos[0] // face_size]
        facerow = next_pos[0] % face_size
        if face == 'B':
            print("B->D")
            next_pos[0], next_pos[1] = rel2abs('D', (face_size-facerow-1, face_size-1))
            next_pos[2] = '<'
            assert what_face(next_pos, face_size) == 'D'
        elif face == 'C':
            print("C->B")
            next_pos[0], next_pos[1] = rel2abs('B', (face_size-1, facerow))
            next_pos[2] = '^'
            assert what_face(next_pos, face_size) == 'B'
        elif face == 'D':
            print("D->B")
            next_pos[0], next_pos[1] = rel2abs('B', (face_size-facerow-1, face_size-1))
            next_pos[2] = '<'
            assert what_face(next_pos, face_size) == 'B'
        elif face == 'F':
            print("F->B")
            next_pos[0], next_pos[1] = rel2abs('D', (face_size-1, facerow))
            next_pos[2] = '^'
            assert what_face(next_pos, face_size) == 'D'
    
    print(next_pos)
    assert 0 <= next_pos[0]
    assert next_pos[0] < face_size*4
    assert 0 <= next_pos[1]
    assert next_pos[1] < face_size*3
    
    return tuple(next_pos)

calc_next_pos([9, 149, '>'], 50)

B->D
[140, 99, '<']


(140, 99, '<')

In [112]:
rotate = {
    'L': { '>': '^', '^': '<', '<': 'v', 'v': '>'},
    'R': { '>': 'v', 'v': '<', '<': '^', '^': '>'},
}
points = {'>': 0, 'v': 1, '<': 2, '^': 3}

def part2(puzzle_input, test=False):
    map, notes, starting_pos = read_input(puzzle_input)
    current_pos = (starting_pos[0], starting_pos[1], '>')
    print("Starting at", current_pos)
    for action in notes:
        if action in ('R', 'L'):
            current_pos = (
                current_pos[0],
                current_pos[1],
                rotate.get(action).get(current_pos[2])
            )
        else:
            
            next_pos = current_pos
            while action > 0:
                if test:
                    next_pos = calc_next_pos_test_input(next_pos, 4)
                else:
                    next_pos = calc_next_pos(next_pos, 50)

                if map[next_pos[0], next_pos[1]] == -1:
                    # hit a wall
                    break
                elif map[next_pos[0], next_pos[1]] == 1:
                    # valid move
                    action -= 1
                    current_pos = next_pos
                # else in the voide

            print("now at", current_pos)

    print("Final Position", current_pos)
    return (current_pos[0] + 1) * 1000 + ((current_pos[1] + 1) * 4) + points.get(current_pos[2])

        

def test_part2():
    ans = part2(os.path.join(os.path.join("..", "dat", "day22_test.txt")), test=True)
    print(ans)
    assert ans == 5031

#test_part2()

ans = part2(os.path.join(os.path.join("..", "dat", "day22.txt")))
print(ans) # 162070 is too low


200 5631
Starting at (0, 50, '>')
[0, 51, '>']
[0, 52, '>']
[0, 53, '>']
[0, 54, '>']
[0, 55, '>']
[0, 56, '>']
[0, 57, '>']
[0, 58, '>']
[0, 59, '>']
[0, 60, '>']
[0, 61, '>']
[0, 62, '>']
[0, 63, '>']
[0, 64, '>']
[0, 65, '>']
[0, 66, '>']
[0, 67, '>']
[0, 68, '>']
now at (0, 67, '>')
[1, 67, 'v']
now at (0, 67, 'v')
[0, 68, '>']
now at (0, 67, '>')
[1, 67, 'v']
now at (0, 67, 'v')
[0, 66, '<']
[0, 65, '<']
[0, 64, '<']
[0, 63, '<']
[0, 62, '<']
[0, 61, '<']
[0, 60, '<']
[0, 59, '<']
[0, 58, '<']
[0, 57, '<']
[0, 56, '<']
[0, 55, '<']
[0, 54, '<']
[0, 53, '<']
[0, 52, '<']
[0, 51, '<']
[0, 50, '<']
[0, 49, '<']
[0, 48, '<']
[0, 47, '<']
[0, 46, '<']
[0, 45, '<']
[0, 44, '<']
[0, 43, '<']
[0, 42, '<']
[0, 41, '<']
[0, 40, '<']
[0, 39, '<']
[0, 38, '<']
[0, 37, '<']
[0, 36, '<']
[0, 35, '<']
[0, 34, '<']
[0, 33, '<']
[0, 32, '<']
[0, 31, '<']
[0, 30, '<']
[0, 29, '<']
[0, 28, '<']
[0, 27, '<']
[0, 26, '<']
[0, 25, '<']
[0, 24, '<']
[0, 23, '<']
[0, 22, '<']
[0, 21, '<']
[0, 20, '<']
[0

In [ ]:
        AAAABBBB
        AAAABBBB
        AAAABBBB
        AAAABBBB
        CCCC
        CCCC
        CCCC
        CCCC
    EEEEDDDD
    EEEEDDDD
    EEEEDDDD
    EEEEDDDD
    FFFF
    FFFF
    FFFF
    FFFF